In [1]:
import numpy as np
import pandas as pd
import os 
import datetime

In [2]:
def data_reader():
    """read data"""
    
    data_dir = os.path.join('..', 'data') #/Users/siliangchen/Airbnb

    train_path = os.path.join(data_dir, 'train.csv')
    meta_path = os.path.join(data_dir, 'metadata.csv')
    holiday_path = os.path.join(data_dir, 'holidays.csv')
    frequency_path = os.path.join(data_dir, 'submission_frequency.csv')    
    test_path = os.path.join(data_dir, 'submission_format.csv')        
    weather_path = os.path.join(data_dir, 'weather_cleaned.csv')

    train = pd.read_csv(train_path)
    meta = pd.read_csv(meta_path)
    holiday = pd.read_csv(holiday_path)
    frequency = pd.read_csv(frequency_path)
    test = pd.read_csv(test_path)
    weather = pd.read_csv(weather_path)
    
    return train, meta, holiday, frequency, test, weather

In [8]:
def data_transfomer(train, holiday, frequency, test, weather):
    """transform some table format"""
    
    ### Train
    train['Timestamp'] = train['Timestamp'].astype('datetime64[ns]')
    train['Weekday'] = train['Timestamp'].apply(lambda x: x.weekday()) 
    train['Date'] =  train['Timestamp'].apply(lambda x: x.date())  
    train['Value'] = train.groupby('SiteId')['Value'].fillna(method='ffill')
    train = train[train['Value'].notnull()]
    train['Hour'] = train['Timestamp'].apply(lambda x: x.hour)
    train['minute_bin'] = train['Timestamp'].apply(lambda x: x.minute)
    
    ### Test
    test['Timestamp'] = test['Timestamp'].astype('datetime64[ns]')
    
    ### Holiday
    if 'Unnamed: 0' in holiday.columns:
        del holiday['Unnamed: 0']
    holiday['Date'] = holiday['Date'].astype('datetime64[ns]').apply(lambda x: x.date())    
    
    ### Frequency
    frequency['ForecastPeriodNS'] = frequency['ForecastPeriodNS'].map({900000000000: '15mins', 
                                                                      3600000000000: '1hour',
                                                                      86400000000000: '1day'})
    
    ### Weather    
    weather['Timestamp'] = weather['Timestamp'].astype('datetime64[ns]')
    weather['Date'] = weather['Timestamp'].apply(lambda x: x.date()) 
    weather['Hour'] = weather['Timestamp'].apply(lambda x: x.hour)
    weather['Minute'] = weather['Timestamp'].apply(lambda x: x.minute) 
    
    # bin minutes into four buckets
    weather['minute_bin'] = pd.cut(weather['Minute'], [-0.1, 7.5, 22.5, 37.5, 52.5, 60], labels=['0','15','30','45','0_2nd'])
    weather.loc[weather.minute_bin == '0_2nd', 'minute_bin'] = '0'
    weather['minute_bin']=weather['minute_bin'].astype(int)
            
    return train, holiday, frequency, test, weather

In [4]:
def data_merger(train, meta, holiday, frequency):
    """merge multiple tables together"""    

    # create onoff feature
    onoff = meta[['SiteId','FridayIsDayOff','SaturdayIsDayOff','SundayIsDayOff']].melt('SiteId').rename(columns={'variable':'Weekday','value':'OnAndOff'})
    onoff['Weekday'] = onoff['Weekday'].map({'FridayIsDayOff': 4, 
                                             'SaturdayIsDayOff': 5,
                                             'SundayIsDayOff': 6})
    # drop unneeded columns in meta
    meta = meta.drop(['MondayIsDayOff','TuesdayIsDayOff','WednesdayIsDayOff','ThursdayIsDayOff','FridayIsDayOff','SaturdayIsDayOff','SundayIsDayOff'], axis=1)

    # merge tables
    result = pd.merge(train, frequency,  on='ForecastId', how='left')
    result = pd.merge(result, holiday,  on=['SiteId','Date'], how='left')
    result = pd.merge(result, onoff,  on=['SiteId','Weekday'], how='left')
    result = pd.merge(result, meta,  on='SiteId', how='left')

    # create isHoliday column
    result['isHoliday'] = result['Holiday'].notnull()

    # fill OnAndOff with False when weekday in 0-3 (Monday to Thursday) 
    result.loc[result.Weekday.isin([0,1,2,3]) , 'OnAndOff'] = False

    return result

In [13]:
def weather_aggregator(weather):
    """aggregate weather info by three different the length of time"""
    
    # get the weather info by site and date
    weather_1day = weather.groupby(['SiteId','Date']).agg({'Weather':np.mean}).reset_index()
    
    # get the weather info by site and hour
    weather_1hour = weather.groupby(['SiteId','Date','Hour']).agg({'Weather':np.mean}).reset_index()
    
    # get the weather info by site and 15 min
    weather_15min = weather.groupby(['SiteId','Date','Hour', 'minute_bin']).agg({'Weather':np.mean}).reset_index()
    
    return weather_1day, weather_1hour, weather_15min  

In [114]:
def train_weather_merger(train_15min, train_1hour, train_1day, weather_15min, weather_1hour, weather_1day):
    """merge train with weather info"""
        
    train_15min = pd.merge(train_15min, weather_15min, on=['SiteId', 'Date', 'Hour', 'minute_bin'], how='left')
    train_1hour = pd.merge(train_1hour, weather_1hour, on=['SiteId', 'Date', 'Hour'], how='left')
    train_1day = pd.merge(train_1day, weather_1day, on=['SiteId', 'Date'], how='left')
    
    return train_15min, train_1hour, train_1day

In [119]:
def fill_weather_with_prev_value(train):
    """fill weather na with previous value by siteid"""
   
    train['Weather'] = train.groupby('SiteId')['Weather'].fillna(method='ffill')
    # train = train[train['Weather'].notnull()]

    return train

In [6]:
# read data
train, meta, holiday, frequency, test, weather = data_reader()

In [9]:
# transform and create new columns
train, holiday, frequency, test, weather = data_transfomer(train, holiday, frequency, test, weather)

/Users/johnnychiu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/johnnychiu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [14]:
# aggregate weather info by three different the length of time
weather_1day, weather_1hour, weather_15min   = weather_aggregator(weather)

In [18]:
# merge tables
train_final = data_merger(train, meta, holiday, frequency)

In [19]:
# sanity check
train_final.isnull().sum()/train_final.shape[0]

obs_id              0.000000
SiteId              0.000000
Timestamp           0.000000
ForecastId          0.000000
Value               0.000000
Weekday             0.000000
Date                0.000000
Hour                0.000000
minute_bin          0.000000
ForecastPeriodNS    0.000000
Holiday             0.975809
OnAndOff            0.000000
Surface             0.000000
Sampling            0.000000
BaseTemperature     0.000000
isHoliday           0.000000
dtype: float64

In [20]:
train_final.head()

,obs_id,SiteId,Timestamp,ForecastId,Value,Weekday,Date,Hour,minute_bin,ForecastPeriodNS,Holiday,OnAndOff,Surface,Sampling,BaseTemperature,isHoliday
0,744519,1,2014-09-03,1,9.096555e+05,2,2014-09-03,0,0,1day,NaN,False,1387.205119,15.0,18.0,False
1,7627564,1,2014-09-04,1,1.748273e+06,3,2014-09-04,0,0,1day,NaN,False,1387.205119,15.0,18.0,False
2,7034705,1,2014-09-05,1,1.748273e+06,4,2014-09-05,0,0,1day,NaN,False,1387.205119,15.0,18.0,False
3,5995486,1,2014-09-06,1,1.748273e+06,5,2014-09-06,0,0,1day,NaN,True,1387.205119,15.0,18.0,False
4,7326510,1,2014-09-07,1,1.748273e+06,6,2014-09-07,0,0,1day,NaN,True,1387.205119,15.0,18.0,False


In [112]:
# generate train by three different the length of time
train_15min = train_final[train_final.ForecastPeriodNS=='15mins']
train_1hour = train_final[train_final.ForecastPeriodNS=='1hour']
train_1day = train_final[train_final.ForecastPeriodNS=='1day']

In [115]:
# merging with weather info
train_15min, train_1hour, train_1day = train_weather_merger(train_15min, train_1hour, train_1day, weather_15min, weather_1hour, weather_1day)

In [130]:
train_15min.head()

,obs_id,SiteId,Timestamp,ForecastId,Value,Weekday,Date,Hour,minute_bin,ForecastPeriodNS,Holiday,OnAndOff,Surface,Sampling,BaseTemperature,isHoliday,Weather
0,2739872,5,2013-01-01 01:00:00,42,24190.917712,1,2013-01-01,1,0,15mins,NaN,False,12541.181277,30.0,18.0,False,NaN
1,4126026,5,2013-01-01 01:15:00,42,24095.677878,1,2013-01-01,1,15,15mins,NaN,False,12541.181277,30.0,18.0,False,NaN
2,6475636,5,2013-01-01 01:30:00,42,24000.438045,1,2013-01-01,1,30,15mins,NaN,False,12541.181277,30.0,18.0,False,NaN
3,2677615,5,2013-01-01 01:45:00,42,24032.184656,1,2013-01-01,1,45,15mins,NaN,False,12541.181277,30.0,18.0,False,NaN
4,6471592,5,2013-01-01 02:00:00,42,24063.931267,1,2013-01-01,2,0,15mins,NaN,False,12541.181277,30.0,18.0,False,NaN


In [116]:
train_15min.Weather.notnull().value_counts()

False    3471030
True     1779590
Name: Weather, dtype: int64

In [131]:
train_1hour.head()

,obs_id,SiteId,Timestamp,ForecastId,Value,Weekday,Date,Hour,minute_bin,ForecastPeriodNS,Holiday,OnAndOff,Surface,Sampling,BaseTemperature,isHoliday,Weather
0,4382312,2,2013-01-01 01:00:00,5,30159.963708,1,2013-01-01,1,0,1hour,NaN,False,6098.278376,30.0,18.0,False,NaN
1,2016541,2,2013-01-01 02:00:00,5,29793.536720,1,2013-01-01,2,0,1hour,NaN,False,6098.278376,30.0,18.0,False,NaN
2,78869,2,2013-01-01 03:00:00,5,31682.412161,1,2013-01-01,3,0,1hour,NaN,False,6098.278376,30.0,18.0,False,NaN
3,1361014,2,2013-01-01 04:00:00,5,29881.829185,1,2013-01-01,4,0,1hour,NaN,False,6098.278376,30.0,18.0,False,NaN
4,129169,2,2013-01-01 05:00:00,5,30560.328940,1,2013-01-01,5,0,1hour,NaN,False,6098.278376,30.0,18.0,False,NaN


In [117]:
train_1hour.Weather.notnull().value_counts()

True     950101
False    310032
Name: Weather, dtype: int64

In [132]:
train_1day.head()

,obs_id,SiteId,Timestamp,ForecastId,Value,Weekday,Date,Hour,minute_bin,ForecastPeriodNS,Holiday,OnAndOff,Surface,Sampling,BaseTemperature,isHoliday,Weather
0,744519,1,2014-09-03,1,9.096555e+05,2,2014-09-03,0,0,1day,NaN,False,1387.205119,15.0,18.0,False,20.752083
1,7627564,1,2014-09-04,1,1.748273e+06,3,2014-09-04,0,0,1day,NaN,False,1387.205119,15.0,18.0,False,22.418750
2,7034705,1,2014-09-05,1,1.748273e+06,4,2014-09-05,0,0,1day,NaN,False,1387.205119,15.0,18.0,False,25.734615
3,5995486,1,2014-09-06,1,1.748273e+06,5,2014-09-06,0,0,1day,NaN,True,1387.205119,15.0,18.0,False,19.635417
4,7326510,1,2014-09-07,1,1.748273e+06,6,2014-09-07,0,0,1day,NaN,True,1387.205119,15.0,18.0,False,17.133333


In [118]:
train_1day.Weather.notnull().value_counts()

True     40706
False    12154
Name: Weather, dtype: int64

In [ ]:
# fill NA with previous weather value
train_15min = fill_weather_with_prev_value(train_15min)
train_1hour = fill_weather_with_prev_value(train_1hour)
train_1day = fill_weather_with_prev_value(train_1day)

In [122]:
train_15min.Weather.notnull().value_counts()
# 26% NA

True     3887551
False    1363069
Name: Weather, dtype: int64

In [123]:
train_1hour.Weather.notnull().value_counts() 
# 19% NA

True     1015359
False     244774
Name: Weather, dtype: int64

In [124]:
train_1day.Weather.notnull().value_counts()
# 18% NA

True     43382
False     9478
Name: Weather, dtype: int64

In [129]:
train_15min.to_csv('../data/train_15min.csv', sep=',', index = False)
train_1hour.to_csv('../data/train_1hour.csv', sep=',', index = False)
train_1day.to_csv('../data/train_1day.csv', sep=',', index = False)

In [128]:
# sanity check
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
print_full(train_15min[(train_15min.SiteId==5)][['Timestamp','Weather']])

              Timestamp  Weather
0   2013-01-01 01:00:00      NaN
1   2013-01-01 01:15:00      NaN
2   2013-01-01 01:30:00      NaN
3   2013-01-01 01:45:00      NaN
4   2013-01-01 02:00:00      NaN
5   2013-01-01 02:15:00      NaN
6   2013-01-01 02:30:00      NaN
7   2013-01-01 02:45:00      NaN
8   2013-01-01 03:00:00      NaN
9   2013-01-01 03:15:00      NaN
10  2013-01-01 03:30:00      NaN
11  2013-01-01 03:45:00      NaN
12  2013-01-01 04:00:00      NaN
13  2013-01-01 04:15:00      NaN
14  2013-01-01 04:30:00      NaN
15  2013-01-01 04:45:00      NaN
16  2013-01-01 05:00:00      NaN
17  2013-01-01 05:15:00      NaN
18  2013-01-01 05:30:00      NaN
19  2013-01-01 05:45:00      NaN
20  2013-01-01 06:00:00      NaN
21  2013-01-01 06:15:00      NaN
22  2013-01-01 06:30:00      NaN
23  2013-01-01 06:45:00      NaN
24  2013-01-01 07:00:00      NaN
25  2013-01-01 07:15:00      NaN
26  2013-01-01 07:30:00      NaN
27  2013-01-01 07:45:00      NaN
28  2013-01-01 08:00:00      NaN
29  2013-0

---

In [133]:
train_unique=train_final.drop_duplicates(subset=['ForecastId'], keep="first")

In [134]:
train_unique.ForecastPeriodNS.value_counts()/train_unique.shape[0]

15mins    0.780614
1hour     0.187267
1day      0.032119
Name: ForecastPeriodNS, dtype: float64

## Other messy code

In [ ]:
# fl_ma['ValueLag1'] = fl_ma.groupby(['ForecastId'])['Value'].shift(1)
# fl_ma['ValueLag2'] = fl_ma.groupby(['ForecastId'])['Value'].shift(2)
# fl_ma['ValueLag3'] = fl_ma.groupby(['ForecastId'])['Value'].shift(3)
# fl_ma.head()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

In [ ]:
# a = ma_df.query("ForecastId==1")[['Timestamp','Value']]
# a.set_index('Timestamp')
# a.plot()

In [ ]:
# lag=10
# for index, row in ma_df.query("ForecastId==1").iterrows():
#     if row['isTrain']==False:
#         ma_df.loc[index, 'Value'] = ma_df.loc[index-lag:index-1,'Value'].mean()

In [ ]:
# b = ma_df.query("ForecastId==1")[['Timestamp','Value']]
# b.set_index('Timestamp')
# b.plot()

